In [46]:
import pandas as pd
import numpy as np
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import export_graphviz
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix


import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

from acquire import get_telco_data

In [2]:
df = get_telco_data()

In [3]:
df.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,monthly_charges,total_charges,phone_service,multiple_lines,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,contract_type,payment_type,internet_service_type,churn
0,0003-MKNFE,Male,0,No,No,9,59.9,542.4,Yes,Yes,...,No,No,No,No,Yes,No,Month-to-month,Mailed check,DSL,No
1,0004-TLHLJ,Male,0,No,No,4,73.9,280.85,Yes,No,...,No,Yes,No,No,No,Yes,Month-to-month,Electronic check,Fiber optic,Yes
2,0011-IGKFF,Male,1,Yes,No,13,98.0,1237.85,Yes,No,...,Yes,Yes,No,Yes,Yes,Yes,Month-to-month,Electronic check,Fiber optic,Yes
3,0013-EXCHZ,Female,1,Yes,No,3,83.9,267.4,Yes,No,...,No,No,Yes,Yes,No,Yes,Month-to-month,Mailed check,Fiber optic,Yes
4,0013-MHZWF,Female,0,No,Yes,9,69.4,571.45,Yes,No,...,No,No,Yes,Yes,Yes,Yes,Month-to-month,Credit card (automatic),DSL,No


In [4]:
df.shape

(7043, 21)

In [5]:
# Check for nulls in the columns of interest
assert df.total_charges.isna().sum() == 0, "There are missing values in this column. Handle them before proceeding"
assert df.monthly_charges.isna().sum() == 0, "There are missing values in this column. Handle them before proceeding"
assert df.tenure.isna().sum() == 0, "There are missing values in this column. Handle them before proceeding"

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
customer_id              7043 non-null object
gender                   7043 non-null object
senior_citizen           7043 non-null int64
partner                  7043 non-null object
dependents               7043 non-null object
tenure                   7043 non-null int64
monthly_charges          7043 non-null float64
total_charges            7043 non-null object
phone_service            7043 non-null object
multiple_lines           7043 non-null object
online_security          7043 non-null object
online_backup            7043 non-null object
device_protection        7043 non-null object
tech_support             7043 non-null object
streaming_tv             7043 non-null object
streaming_movies         7043 non-null object
paperless_billing        7043 non-null object
contract_type            7043 non-null object
payment_type             7043 non-null object
internet_service_typ

In [7]:
#why is total_charges an object?
df.total_charges.value_counts(dropna=False)

20.2       11
           11
19.75       9
20.05       8
19.65       8
           ..
39.3        1
4275.75     1
5791.1      1
187.45      1
2414.55     1
Name: total_charges, Length: 6531, dtype: int64

- It looks like there are 11 observations where the total_charges is an empty string
- Need to change total_charges to a float

In [8]:
# Make any string with multiple spaces into an empty space value
df.total_charges = df.total_charges.str.strip()

# Count the number of empty string entries
df[df.total_charges == ""]

,customer_id,gender,senior_citizen,partner,dependents,tenure,monthly_charges,total_charges,phone_service,multiple_lines,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,contract_type,payment_type,internet_service_type,churn
1475,1371-DWPAZ,Female,0,Yes,Yes,0,56.05,,No,No phone service,...,Yes,Yes,Yes,Yes,No,No,Two year,Credit card (automatic),DSL,No
2670,2923-ARZLG,Male,0,Yes,Yes,0,19.70,,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Yes,One year,Mailed check,None,No
2942,2520-SGTTA,Female,0,Yes,Yes,0,20.00,,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,No,Two year,Mailed check,None,No
2979,2775-SEFEE,Male,0,No,Yes,0,61.90,,Yes,Yes,...,Yes,No,Yes,No,No,Yes,Two year,Bank transfer (automatic),DSL,No
3031,3115-CZMZD,Male,0,No,Yes,0,20.25,,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,No,Two year,Mailed check,None,No
3050,3213-VVOLG,Male,0,Yes,Yes,0,25.35,,Yes,Yes,...,No internet service,No internet service,No internet service,No internet service,No internet service,No,Two year,Mailed check,None,No
3204,4075-WKNIU,Female,0,Yes,Yes,0,73.35,,Yes,Yes,...,Yes,Yes,Yes,Yes,No,No,Two year,Mailed check,DSL,No
3242,4367-NUYAO,Male,0,Yes,Yes,0,25.75,,Yes,Yes,...,No internet service,No internet service,No internet service,No internet service,No internet service,No,Two year,Mailed check,None,No
3252,4472-LVYGI,Female,0,Yes,Yes,0,52.55,,No,No phone service,...,No,Yes,Yes,Yes,No,Yes,Two year,Bank transfer (automatic),DSL,No
4710,5709-LVOEQ,Female,0,Yes,Yes,0,80.85,,Yes,No,...,Yes,Yes,No,Yes,Yes,No,Two year,Mailed check,DSL,No


It looks like those values are blank bc the tenure is 0. I will change it to 1 since they were probably customers for about a month

In [9]:
df.tenure.value_counts().sort_index()

0      11
1     613
2     238
3     200
4     176
     ... 
68    100
69     95
70    119
71    170
72    362
Name: tenure, Length: 73, dtype: int64

In [10]:
# Replace any tenures of 0 with 1

df.tenure.replace(0, 1, inplace=True)
# # Validate my tenure count for value 1

df.tenure.value_counts().sort_index()

1     624
2     238
3     200
4     176
5     133
     ... 
68    100
69     95
70    119
71    170
72    362
Name: tenure, Length: 72, dtype: int64

In [11]:
df[df.tenure == 1]

,customer_id,gender,senior_citizen,partner,dependents,tenure,monthly_charges,total_charges,phone_service,multiple_lines,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,contract_type,payment_type,internet_service_type,churn
7,0021-IKXGC,Female,1,No,No,1,72.10,72.1,Yes,Yes,...,No,No,No,No,No,Yes,Month-to-month,Electronic check,Fiber optic,No
8,0023-HGHWL,Male,1,No,No,1,25.10,25.1,No,No phone service,...,No,No,No,No,No,Yes,Month-to-month,Electronic check,DSL,Yes
13,0032-PGELS,Female,0,Yes,Yes,1,30.50,30.5,No,No phone service,...,No,No,No,No,No,No,Month-to-month,Bank transfer (automatic),DSL,Yes
19,0082-LDZUE,Male,0,No,No,1,44.30,44.3,Yes,No,...,No,No,No,No,No,Yes,Month-to-month,Mailed check,DSL,No
29,0107-WESLM,Male,0,No,No,1,19.85,19.85,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Yes,Month-to-month,Electronic check,None,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6794,9907-SWKKF,Female,1,No,No,1,25.05,25.05,No,No phone service,...,No,No,No,No,No,No,Month-to-month,Mailed check,DSL,Yes
6808,9940-RHLFB,Female,0,No,No,1,75.30,75.3,Yes,No,...,No,Yes,No,No,No,No,Month-to-month,Electronic check,Fiber optic,Yes
6816,9962-BFPDU,Female,0,Yes,Yes,1,20.05,20.05,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,No,Month-to-month,Mailed check,None,No
6823,9975-SKRNR,Male,0,No,No,1,18.90,18.9,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,No,Month-to-month,Mailed check,None,No


In [12]:
# Replace the blank total_charges with the monthly_charge for tenure == 1
df.total_charges.replace('', df.monthly_charges, inplace=True)

In [13]:
# Validate my changes
df[df.tenure == 1]

,customer_id,gender,senior_citizen,partner,dependents,tenure,monthly_charges,total_charges,phone_service,multiple_lines,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,contract_type,payment_type,internet_service_type,churn
7,0021-IKXGC,Female,1,No,No,1,72.10,72.1,Yes,Yes,...,No,No,No,No,No,Yes,Month-to-month,Electronic check,Fiber optic,No
8,0023-HGHWL,Male,1,No,No,1,25.10,25.1,No,No phone service,...,No,No,No,No,No,Yes,Month-to-month,Electronic check,DSL,Yes
13,0032-PGELS,Female,0,Yes,Yes,1,30.50,30.5,No,No phone service,...,No,No,No,No,No,No,Month-to-month,Bank transfer (automatic),DSL,Yes
19,0082-LDZUE,Male,0,No,No,1,44.30,44.3,Yes,No,...,No,No,No,No,No,Yes,Month-to-month,Mailed check,DSL,No
29,0107-WESLM,Male,0,No,No,1,19.85,19.85,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,Yes,Month-to-month,Electronic check,None,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6794,9907-SWKKF,Female,1,No,No,1,25.05,25.05,No,No phone service,...,No,No,No,No,No,No,Month-to-month,Mailed check,DSL,Yes
6808,9940-RHLFB,Female,0,No,No,1,75.30,75.3,Yes,No,...,No,Yes,No,No,No,No,Month-to-month,Electronic check,Fiber optic,Yes
6816,9962-BFPDU,Female,0,Yes,Yes,1,20.05,20.05,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,No,Month-to-month,Mailed check,None,No
6823,9975-SKRNR,Male,0,No,No,1,18.90,18.9,Yes,No,...,No internet service,No internet service,No internet service,No internet service,No internet service,No,Month-to-month,Mailed check,None,No


In [14]:
df.total_charges = df.total_charges.astype(float)

In [15]:
def wrangle_telco():
    df = acquire.get_telco_data()
    df.tenure.replace(0, 1, inplace=True)
    df.total_charges = df.total_charges.str.strip()
    df.total_charges.replace('', df.monthly_charges, inplace=True)
    df.total_charges = df.total_charges.astype(float)
    return df

In [16]:
# step 1 split
train, test = train_test_split(df, random_state=123, train_size=.86)
train, validate = train_test_split(train, random_state=123, train_size=.83)

In [17]:
# step 2 for loop w/ list of columns to encode
encoder = LabelEncoder()

encode_list = [
    'gender', 'partner', 'dependents', 'phone_service'
    , 'multiple_lines', 'online_security', 'online_backup'
    , 'device_protection', 'tech_support'
    , 'streaming_movies', 'streaming_tv', 'paperless_billing', 'churn'
    ]
               
               
               
for e in encode_list:
    train[e] = encoder.fit_transform(train[e])
    test[e] = encoder.transform(test[e])

In [18]:
train.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,monthly_charges,total_charges,phone_service,multiple_lines,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,contract_type,payment_type,internet_service_type,churn
4782,6119-SPUDB,1,0,0,0,46,38.25,1755.35,0,1,...,0,2,2,0,0,0,Two year,Mailed check,DSL,0
3552,5309-TAIKL,0,0,0,0,7,62.80,418.30,1,0,...,0,2,2,0,2,0,Month-to-month,Bank transfer (automatic),DSL,0
1688,2516-VQRRV,0,1,0,0,2,75.45,158.40,1,2,...,0,0,0,0,0,1,Month-to-month,Mailed check,Fiber optic,1
3210,4111-BNXIF,0,0,1,1,67,59.55,4103.90,0,1,...,2,2,2,2,2,1,Two year,Electronic check,DSL,0
2647,2809-ILCYT,0,0,1,0,25,26.80,733.55,1,2,...,1,1,1,1,1,1,One year,Mailed check,None,0


In [19]:
train.shape

(5026, 21)

In [20]:
test.shape

(987, 21)

In [21]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5026 entries, 4782 to 4315
Data columns (total 21 columns):
customer_id              5026 non-null object
gender                   5026 non-null int64
senior_citizen           5026 non-null int64
partner                  5026 non-null int64
dependents               5026 non-null int64
tenure                   5026 non-null int64
monthly_charges          5026 non-null float64
total_charges            5026 non-null float64
phone_service            5026 non-null int64
multiple_lines           5026 non-null int64
online_security          5026 non-null int64
online_backup            5026 non-null int64
device_protection        5026 non-null int64
tech_support             5026 non-null int64
streaming_tv             5026 non-null int64
streaming_movies         5026 non-null int64
paperless_billing        5026 non-null int64
contract_type            5026 non-null object
payment_type             5026 non-null object
internet_service_type    502

In [22]:
# How many people have churned?
train.churn.value_counts()

0    3714
1    1312
Name: churn, dtype: int64

Let's try to narrow down which features have an effect on churn
try a .corr 

In [23]:
train.corr()

,gender,senior_citizen,partner,dependents,tenure,monthly_charges,total_charges,phone_service,multiple_lines,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,churn
gender,1.000000,0.014134,-0.006958,0.006573,-0.000597,-0.015672,-0.011372,-0.000349,-0.008863,-0.008304,-0.013978,0.000548,-0.003688,-0.010969,-0.017253,-0.009805,-0.007882
senior_citizen,0.014134,1.000000,0.024662,-0.215656,0.017738,0.229326,0.105798,0.012264,0.141595,-0.127436,-0.017425,-0.023220,-0.154157,0.035651,0.042131,0.173264,0.165401
partner,-0.006958,0.024662,1.000000,0.449464,0.363586,0.092680,0.303653,0.013391,0.138616,0.157758,0.144962,0.159423,0.114294,0.134285,0.131481,-0.018159,-0.135911
dependents,0.006573,-0.215656,0.449464,1.000000,0.151871,-0.121235,0.050836,-0.003816,-0.021338,0.148760,0.095236,0.069991,0.119919,0.047913,0.022778,-0.116526,-0.145364
tenure,-0.000597,0.017738,0.363586,0.151871,1.000000,0.259495,0.827749,0.011914,0.352729,0.327803,0.375374,0.364255,0.327208,0.304074,0.292760,-0.002013,-0.347336
monthly_charges,-0.015672,0.229326,0.092680,-0.121235,0.259495,1.000000,0.657768,0.245498,0.427001,-0.043797,0.126294,0.173123,0.002349,0.346986,0.337990,0.353444,0.200578
total_charges,-0.011372,0.105798,0.303653,0.050836,0.827749,0.657768,1.000000,0.114976,0.453621,0.256213,0.380937,0.387860,0.280976,0.407728,0.395868,0.151714,-0.192194
phone_service,-0.000349,0.012264,0.013391,-0.003816,0.011914,0.245498,0.114976,1.000000,-0.016459,-0.018330,0.025620,0.019842,-0.013332,0.057677,0.045825,0.016024,0.013591
multiple_lines,-0.008863,0.141595,0.138616,-0.021338,0.352729,0.427001,0.453621,-0.016459,1.000000,0.010194,0.123864,0.116567,0.009751,0.187262,0.179967,0.161766,0.041354
online_security,-0.008304,-0.127436,0.157758,0.148760,0.327803,-0.043797,0.256213,-0.018330,0.010194,1.000000,0.191167,0.174618,0.280722,0.059718,0.063086,-0.162925,-0.281175


Which contract type has the highest number of people churning?

In [24]:
train.groupby(['churn']).contract_type.value_counts()

churn  contract_type 
0      Month-to-month    1593
       Two year          1178
       One year           943
1      Month-to-month    1168
       One year           108
       Two year            36
Name: contract_type, dtype: int64

In [25]:
train.groupby(['contract_type', 'churn']).tenure.count().nlargest()

contract_type   churn
Month-to-month  0        1593
Two year        0        1178
Month-to-month  1        1168
One year        0         943
                1         108
Name: tenure, dtype: int64

- It looks like the highest number of people who have churned have a month to month contract and they tend to churn just after the first month.
- why?

In [26]:
#Does the churn group buy more of one specific type of internet?
train.groupby(['churn']).internet_service_type.value_counts()

churn  internet_service_type
0      DSL                      1414
       Fiber optic              1271
       None                     1029
1      Fiber optic               916
       DSL                       318
       None                       78
Name: internet_service_type, dtype: int64

In [27]:
train.groupby(['churn']).payment_type.value_counts()

churn  payment_type             
0      Mailed check                 953
       Electronic check             927
       Bank transfer (automatic)    921
       Credit card (automatic)      913
1      Electronic check             748
       Mailed check                 214
       Credit card (automatic)      176
       Bank transfer (automatic)    174
Name: payment_type, dtype: int64

In [28]:
train.groupby(['churn','contract_type']).payment_type.value_counts()

churn  contract_type   payment_type             
0      Month-to-month  Electronic check             616
                       Mailed check                 445
                       Bank transfer (automatic)    277
                       Credit card (automatic)      255
       One year        Bank transfer (automatic)    256
                       Credit card (automatic)      252
                       Mailed check                 234
                       Electronic check             201
       Two year        Credit card (automatic)      406
                       Bank transfer (automatic)    388
                       Mailed check                 274
                       Electronic check             110
1      Month-to-month  Electronic check             696
                       Mailed check                 197
                       Bank transfer (automatic)    140
                       Credit card (automatic)      135
       One year        Electronic check              41

- It looks like the non-automatic payment types account for the highest number of people who have churned.
- I would like to do some feature engineering that groups payment type into automatic and manual

In [29]:
df1 = train[['churn','contract_type', 'payment_type']]

In [30]:
df1.head()

,churn,contract_type,payment_type
4782,0,Two year,Mailed check
3552,0,Month-to-month,Bank transfer (automatic)
1688,1,Month-to-month,Mailed check
3210,0,Two year,Electronic check
2647,0,One year,Mailed check


In [31]:
melt = df1.melt(id_vars=['churn','contract_type'], value_name='payment_mode')
melt.index=train.index

In [32]:
melt.head()

,churn,contract_type,variable,payment_mode
4782,0,Two year,payment_type,Mailed check
3552,0,Month-to-month,payment_type,Bank transfer (automatic)
1688,1,Month-to-month,payment_type,Mailed check
3210,0,Two year,payment_type,Electronic check
2647,0,One year,payment_type,Mailed check


In [33]:
melt = melt['payment_mode'].str.split('(', expand = True)
melt.head()

,0,1
4782,Mailed check,None
3552,Bank transfer,automatic)
1688,Mailed check,None
3210,Electronic check,None
2647,Mailed check,None


In [34]:
melt.columns = ['payment_method', 'mode_of_payment']

In [35]:
df1 = pd.concat([df1, melt], axis=1)
df1.head()

,churn,contract_type,payment_type,payment_method,mode_of_payment
4782,0,Two year,Mailed check,Mailed check,None
3552,0,Month-to-month,Bank transfer (automatic),Bank transfer,automatic)
1688,1,Month-to-month,Mailed check,Mailed check,None
3210,0,Two year,Electronic check,Electronic check,None
2647,0,One year,Mailed check,Mailed check,None


In [36]:
df1 = df1.drop(columns='payment_method')
df1.head()

,churn,contract_type,payment_type,mode_of_payment
4782,0,Two year,Mailed check,None
3552,0,Month-to-month,Bank transfer (automatic),automatic)
1688,1,Month-to-month,Mailed check,None
3210,0,Two year,Electronic check,None
2647,0,One year,Mailed check,None


In [37]:
df1['mode_of_payment'] = df1.mode_of_payment.fillna('manual')
df1['mode_of_payment'] = df1.mode_of_payment.str.replace(')','')
df1.head()

,churn,contract_type,payment_type,mode_of_payment
4782,0,Two year,Mailed check,manual
3552,0,Month-to-month,Bank transfer (automatic),automatic
1688,1,Month-to-month,Mailed check,manual
3210,0,Two year,Electronic check,manual
2647,0,One year,Mailed check,manual


In [38]:
df1.groupby('churn').payment_type.value_counts()

churn  payment_type             
0      Mailed check                 953
       Electronic check             927
       Bank transfer (automatic)    921
       Credit card (automatic)      913
1      Electronic check             748
       Mailed check                 214
       Credit card (automatic)      176
       Bank transfer (automatic)    174
Name: payment_type, dtype: int64

In [39]:
df1.mode_of_payment.value_counts()

manual       2842
automatic    2184
Name: mode_of_payment, dtype: int64

In [40]:
df1.groupby(['churn','contract_type']).mode_of_payment.value_counts()

churn  contract_type   mode_of_payment
0      Month-to-month  manual             1061
                       automatic           532
       One year        automatic           508
                       manual              435
       Two year        automatic           794
                       manual              384
1      Month-to-month  manual              893
                       automatic           275
       One year        manual               55
                       automatic            53
       Two year        automatic            22
                       manual               14
Name: mode_of_payment, dtype: int64

In [41]:
df1.groupby(['churn'])['contract_type'].count().reset_index()

,churn,contract_type
0,0,3714
1,1,1312


In [49]:
train.columns

Index(['customer_id', 'gender', 'senior_citizen', 'partner', 'dependents',
       'tenure', 'monthly_charges', 'total_charges', 'phone_service',
       'multiple_lines', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
       'paperless_billing', 'contract_type', 'payment_type',
       'internet_service_type', 'churn'],
      dtype='object')

In [50]:
X = df[['senior_citizen','multiple_lines','streaming_movies','paperless_billing','contract_type','payment_type']]
y = df[['churn']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .30, random_state = 123)

In [52]:
encoder = LabelEncoder()

encode_list = [
    'gender', 'partner', 'dependents', 'phone_service'
    , 'multiple_lines', 'online_security', 'online_backup'
    , 'device_protection', 'tech_support'
    , 'streaming_movies', 'streaming_tv', 'paperless_billing', 'churn'
    ]
               
               
               
for e in encode_list:
    train[e] = encoder.fit_transform(train[e])
    test[e] = encoder.transform(test[e])

In [53]:
train.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,monthly_charges,total_charges,phone_service,multiple_lines,...,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,paperless_billing,contract_type,payment_type,internet_service_type,churn
4782,6119-SPUDB,1,0,0,0,46,38.25,1755.35,0,1,...,0,2,2,0,0,0,Two year,Mailed check,DSL,0
3552,5309-TAIKL,0,0,0,0,7,62.80,418.30,1,0,...,0,2,2,0,2,0,Month-to-month,Bank transfer (automatic),DSL,0
1688,2516-VQRRV,0,1,0,0,2,75.45,158.40,1,2,...,0,0,0,0,0,1,Month-to-month,Mailed check,Fiber optic,1
3210,4111-BNXIF,0,0,1,1,67,59.55,4103.90,0,1,...,2,2,2,2,2,1,Two year,Electronic check,DSL,0
2647,2809-ILCYT,0,0,1,0,25,26.80,733.55,1,2,...,1,1,1,1,1,1,One year,Mailed check,None,0


In [47]:
logit = LogisticRegression()

In [48]:
logit.fit(X_train, y_train)

NameError: name 'X_train' is not defined